# First iteration

## Part 1: Get data and build model

In [8]:
import numpy as np
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt
import itertools
from dateutil.relativedelta import relativedelta
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours
from sklearn.metrics import classification_report, confusion_matrix, precision_score, recall_score, f1_score
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, StratifiedShuffleSplit
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import scale
from sklearn.metrics import roc_auc_score
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from joblib import Parallel, delayed
pd.set_option('display.max_columns', None)

## Data Sets Used

Freddie Mac has created a smaller dataset, which is a random sample of 50,000 loans selected from each full vintage year. Each vintage year has one origination data file and one monthly performance file, containing the same loan-level data fields as those included in the full dataset. We have located the `sample_2016.zip` file from the full dataset package, and used this zip package as our data source for this iteration.

The 2016 zip packages has two files: `sample_orig_2016.txt` and `sample_svcg_2016.txt`. The .txt files do not come with headers but instead, we refer to the User Guide (http://www.freddiemac.com/research/pdf/user_guide.pdf) to grab the name of the columns. We then join the two data files together by the loan number. 

It is expected that as we progressed further, we will be using larger and larger datasets. But for this first iteration, this is what we have chosen.

In [2]:
def get_all_data():
    dir = 'D:\\Backups\\StemData\\'
    files = ['sample_orig_2016.txt', 'sample_orig_2015.txt', 'sample_orig_2014.txt', 'sample_orig_2013.txt',
             'sample_orig_2012.txt', 'sample_orig_2011.txt',
             'sample_orig_2010.txt', 'sample_orig_2009.txt', 'sample_orig_2008.txt', 'sample_orig_2007.txt']

    files1 = ['sample_svcg_2016.txt', 'sample_svcg_2015.txt', 'sample_svcg_2014.txt', 'sample_svcg_2013.txt',
              'sample_svcg_2012.txt', 'sample_svcg_2011.txt',
              'sample_svcg_2010.txt', 'sample_svcg_2009.txt', 'sample_svcg_2008.txt', 'sample_svcg_2008.txt']

    merged_data = pd.DataFrame()
    for i in [0]:
        print(files[i])
        raw = pd.read_csv(dir + files[i], sep='|', header=None, low_memory=False)
        raw.columns = ['credit_score', 'first_pmt_date', 'first_time', 'mat_date', 'msa', 'mi_perc', 'units',
                       'occ_status', 'ocltv', 'odti', 'oupb', 'oltv', 'oint_rate', 'channel', 'ppm', 'fixed_rate',
                       'state', 'prop_type', 'zip', 'loan_num', 'loan_purpose', 'oterm', 'num_borrowers', 'seller_name',
                       'servicer_name', 'exceed_conform']

        raw1 = pd.read_csv(dir + files1[i], sep='|', header=None, low_memory=False)
        raw1.columns = ['loan_num', 'yearmon', 'curr_upb', 'curr_delinq', 'loan_age', 'remain_months', 'repurchased',
                        'modified', 'zero_bal', 'zero_date', 'curr_rate', 'curr_def_upb', 'ddlpi', 'mi_rec',
                        'net_proceeds',
                        'non_mi_rec', 'exp', 'legal_costs', 'maint_exp', 'tax_insur', 'misc_exp', 'loss', 'mod_exp']

        data = pd.merge(raw, raw1, on='loan_num', how='inner')

        merged_data = merged_data.append(data)

    merged_data.drop(['seller_name', 'servicer_name', 'first_pmt_date', 'mat_date', 'msa', 'net_proceeds'], axis=1, inplace=True)

    # all data must have the following: credit_score, ocltv, odti, oltv, oint_rate, curr_upb
    # remove any datapoints with missing values from the above features
    merged_data.dropna(subset=['credit_score', 'odti', 'oltv', 'oint_rate', 'curr_upb'], how='any', inplace=True)
    merged_data.credit_score = pd.to_numeric(data['credit_score'], errors='coerce')
    merged_data.yearmon = pd.to_datetime(data['yearmon'], format='%Y%m')
    merged_data.fillna(value=0, inplace=True, axis=1)
    
    merged_data.sort_values(['loan_num'], ascending=True).groupby(['yearmon'], as_index=False)  ##consider move this into the next func
    merged_data.set_index(['loan_num', 'yearmon'], inplace=True) ## consider move this into the next func

    return merged_data

## Treatment of Missing Values (So Far)

Key features that are missing are more likely to be the result of reporting errors by the originator or the servicer, or incomplete information provided by the borrower. Similar to the Deep Learning paper we are reading, we have insisted that an observation must have no missing values in any of the following:

* FICO score

* LTV ratio

* Original interest rate

* original balance

Samples missing one of these variables are removed. 

After this step, we still have lots of missing values -- a lot of them came from the performance file (such as loan modification costs, legal expenses, etc). Our treatment so far is to treat the missing values as zero, as an missing value of these fields tend to be due to the fact that there hasn't been such an incidence yet.

It is clear that we will need to fine-tune our current treatment of missing values. This will be done in the second iteration by leveraging research already done by other STEM interns.



## Feature Space

Here, we also model after the Deep Learning for Mortgage Risk paper. In the paper, the authors have enumerated the possible states (current, 30 days delinquent, etc), and together, with other loan_level features (listed in Table 2 and Table 6 in the paper), formed the feature space for their model.

We do similar things here. The following code chunk further process the data: 

* Get the delinquency status that is associated with the loans and last observed month, and add a data column called `prev_delin`, in contrast with `curr_delinq`

* Remove the `curr_delinq` from our features but the feature space still has `prev_delinq` variable

* Use `curr_delinq` as our taget

* For the categorical variables, we convert them into dummy/indicator variables


In [3]:
def process_data(data):
    #data.sort_values(['loan_num'], ascending=True).groupby(['yearmon'], as_index=False)  ##consider move this out
    #data.set_index(['loan_num', 'yearmon'], inplace=True) ## consider move this out
    y = data['curr_delinq']
    y = y.apply(lambda x:1 if x not in (0, 1) else 0)
    #data['prev_delinq'] = data.curr_delinq.shift(1) ## needs attention here
    #data['prev_delinq'] = data.groupby(level=0)['curr_delinq'].shift(1)
    #print(sum(data.prev_delinq.isnull()))
    data.fillna(value=0, inplace=True, axis=1)
    data.drop(['curr_delinq'], axis=1, inplace=True)
    print(y.shape)
    ## how many classes are y?
    ## remove y from X
    
    X = pd.get_dummies(data)
    #X.net_proceeds = X.net_proceeds.apply(lambda x:0 if x == 'C' else x)
    #y = label_binarize(y, classes=[0, 1, 2, 3]) ## do we really have to do this?
    X[['credit_score','mi_perc','units','ocltv', 'oupb', 'oltv', 'oint_rate','zip',
       'curr_upb','loan_age','remain_months', 'curr_rate','curr_def_upb', 'ddlpi','mi_rec',
       'non_mi_rec', 'exp', 'legal_costs','maint_exp','tax_insur', 'misc_exp', 'loss','mod_exp']] = \
    scale(X[['credit_score','mi_perc','units','ocltv', 'oupb', 'oltv', 'oint_rate','zip',
       'curr_upb','loan_age','remain_months', 'curr_rate','curr_def_upb', 'ddlpi','mi_rec',
       'non_mi_rec', 'exp', 'legal_costs','maint_exp','tax_insur', 'misc_exp', 'loss','mod_exp']], with_mean=False)
    return X, y

## Getting plotting utility ready

We define the function to plot confusion matrix beow.

In [4]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [5]:
def merge_dict(dicts):
    """dicts: a list of dicts"""
    super_dict = {}
    for d in dicts:
        for k, v in d.items():  # d.items() in Python 3+
            super_dict.setdefault(k, []).append(v)

    df = pd.DataFrame.from_dict(super_dict)
    df.plot()
    plt.show()

    return df

## Decision Tree with Pruning

Decision Trees. For the decision tree, you should implement or steal a decision tree algorithm (and by "implement or steal" I mean "steal"). Be sure to use some form of pruning. You are not required to use information gain (for example, there is something called the GINI index that is sometimes used) to split attributes, but you should describe whatever it is that you do use.

In [ ]:
def complexity_dt(X, y):
    #X_train, y_train, X_test, y_test = train_test_split(X,y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=778)
    #smote = SMOTE(ratio=1)
    #X_train_res, y_train_res = smote.fit_sample(X_train, y_train)
    print('Start Search')
    dt = DecisionTreeClassifier(class_weight='balanced')
    pipe = Pipeline([('dt', dt)])
    param_grid = {'dt__max_depth': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]}
    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=6, cv=5, scoring='neg_log_loss', verbose=5)
    grid_search.fit(X_train, y_train)
    clf = grid_search.best_estimator_
    print('clf', clf)
    print('best_score', grid_search.best_score_)
    y_pred = clf.predict(X_test)
    check_pred = clf.predict(X_train)
    target_names = ['Not delinq', 'Delinq']
    print(classification_report(y_test, y_pred, target_names=target_names))
    conf_mat = confusion_matrix(y_test, y_pred)
    plt.figure()
    plot_confusion_matrix(conf_mat, classes=target_names,
                      title='Confusion matrix, without normalization')
    plt.show()
    return clf, clf.predict(X_train), y_pred

In [ ]:
clf, score, mat = complexity_dt(train, target)

In [ ]:
import scipy
scipy.stats.itemfreq(score)
scipy.stats.itemfreq(mat)

In [ ]:
def train_size_dt(train, target):
    d = {'train': [], 'cv set': [], 'test': []}
    print('here')
    training_features, test_features, \
    training_target, test_target, = train_test_split(train, target, test_size=0.33, random_state=778)
    for size in np.arange(0.1, 1, 0.1):
        print('size', size)
        X_train, X_val, y_train, y_val = train_test_split(training_features, training_target, train_size=size)
        print('start')
        start_time = time.time()
        dt = DecisionTreeClassifier(class_weight='balanced')
        dt.fit(X_train, y_train)
        print('Decision Tree took', time.time() - start_time, 'to run')
        print('process')
        d['train'].append(f1_score(y_train, dt.predict(X_train), average='weighted'))
        d['cv set'].append(f1_score(y_val, dt.predict(X_val), average='weighted'))
        d['test'].append(f1_score(test_target, dt.predict(test_features), average='weighted'))
        print('end')

    return d

The way to use parallel processing in Windows: must wrap this in the main function. 

In [ ]:
all_data = get_all_data()
train, target = process_data(all_data)
df = pd.DataFrame.from_dict(train_size_dt(train=train, target=target))
df.plot()
plt.show()

## Nueral Network Model: First Iteration


We had used the grid search approach to find the the best number of hidden layers (out of 1, 3, 5, and 7). For each of these options, we started out with the full set of features, then reduce it to 70% of that for each subsequent hidden layers.

The authors' deep learning give them a probability transition matrix. 

Our model below gives us a probability matrix for each observation data. This is slightly different.

However, with a bit more work, we can convert our probability matrix produced from our model into the probability transition matrix, so that it not only predicts for us, when a new data comes in, what is the most likely delinquent status of a new loan, but also tell us what is the probability that a loan of a particular delinquency status will transition into a different status type. 

In [ ]:
def complexity_ann(X, y):
    # X_train, y_train, X_test, y_test = train_test_split(X,y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=778)
    # smote = SMOTE(ratio=1)
    # X_train_res, y_train_res = smote.fit_sample(X_train, y_train)
    print('Start Search')
    mlp = MLPClassifier(verbose=True)
    pipe = Pipeline([('mlp', mlp)])
    param_grid = {
        'mlp__hidden_layer_sizes': [(160), (160, 112, 112), (160, 112, 112, 112, 112), (160, 112, 112, 112, 112, 112, 112)]}
    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=6, cv=10, scoring='neg_log_loss', verbose=5)
    grid_search.fit(X_train, y_train)
    clf = grid_search.best_estimator_
    print('clf', clf)
    print('best_score', grid_search.best_score_)
    y_pred = clf.predict(X_test)
    check_pred = clf.predict(X_train)
    target_names = ['Not delinq', 'Delinq']
    print(classification_report(y_test, y_pred, target_names=target_names))
    conf_mat = confusion_matrix(y_test, y_pred)
    plt.figure()
    plot_confusion_matrix(conf_mat, classes=target_names,
                                  title='Confusion matrix, without normalization')
    plt.show()
    return clf, clf.predict(X_train), y_pred

In [ ]:
clf, score, mat = complexity_ann(train, target)

In [ ]:
def train_size_ann(train, target):
    d = {'train': [], 'cv set': [], 'test': []}
    print('here')
    training_features, test_features, \
    training_target, test_target, = train_test_split(train, target, test_size=0.33, random_state=778)
    for size in np.arange(0.1, 1, 0.1):
        print('size', size)
        X_train, X_val, y_train, y_val = train_test_split(training_features, training_target, train_size=size)
        print('start')
        start_time = time.time()
        clf = MLPClassifier()
        clf.fit(X_train, y_train)
        print('Neural Network Tree took', time.time() - start_time, 'to run')
        d['train'].append(f1_score(y_train, clf.predict(X_train), average='weighted'))
        d['cv set'].append(f1_score(y_val, clf.predict(X_val), average='weighted'))
        d['test'].append(f1_score(test_target, clf.predict(test_features), average='weighted'))
        print('end')

    return d

In [ ]:
all_data = get_all_data()
train, target = process_data(all_data)
df = pd.DataFrame.from_dict(train_size_ann(train=train, target=target))
df.plot()
plt.show()

## Boosting

In [ ]:
target.value_counts()

In [ ]:
def complexity_boost(X, y):
    #X_train, y_train, X_test, y_test = train_test_split(X,y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=778)
    smote = SMOTE(ratio=1)
    X_train_res, y_train_res = smote.fit_sample(X_train, y_train)
    print('Start Decision Tree Search')
    boost = GradientBoostingClassifier(n_estimators=100)
    pipe = Pipeline([('smote', smote), ('boost', boost)])
    param_grid = {'boost__max_depth': [1, 2, 3]}
    #sss = StratifiedShuffleSplit(n_splits=500, test_size=0.2)  ## no need for this given 50000 random sample
    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=6, cv=10, scoring='neg_log_loss',verbose=5)
    grid_search.fit(X_train_res, y_train_res)
    clf = grid_search.best_estimator_
    print('clf', clf)
    print('best_score', grid_search.best_score_)
    y_pred = clf.predict(X_test)
    check_pred = clf.predict(X_train)
    target_names = ['Not delinq', 'Delinq']
    print(classification_report(y_test, y_pred, target_names=target_names))
    conf_mat = confusion_matrix(y_test, y_pred)
    plt.figure()
    plot_confusion_matrix(conf_mat, classes=target_names,
                      title='Confusion matrix, without normalization')
    plt.show()    
    return clf, clf.predict(X_train_res), y_pred

In [ ]:
clf, score, mat = complexity_boost(train, target)

In [ ]:
def train_size_boost(train, target, size=0):
    d = {'train': [], 'cv set': [], 'test': []}
    training_features, test_features, \
    training_target, test_target, = train_test_split(train, target, test_size=0.33, random_state=778)
    for size in np.arange(0.1, 1, 0.1):
        print('size', size)
        X_train, X_val, y_train, y_val = train_test_split(training_features, training_target, train_size=size)
        print('start')
        start_time = time.time()
        clf = GradientBoostingClassifier(n_estimators=1000)
        clf.fit(X_train, y_train)
        print('Decision Tree took', time.time() - start_time, 'to run')
        d['train'].append(f1_score(y_train, clf.predict(X_train), average='weighted'))
        d['cv set'].append(f1_score(y_val, clf.predict(X_val), average='weighted'))
        d['test'].append(f1_score(test_target, clf.predict(test_features), average='weighted'))
        print('end')

    return d

In [ ]:
all_data = get_all_data()
train, target = process_data(all_data)
df = pd.DataFrame.from_dict(train_size_boost(train=train, target=target))
df.plot()
plt.show()

## Support Vector Machine

In [ ]:
def complexity_svc(X, y):
    #X_train, y_train, X_test, y_test = train_test_split(X,y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=778)
    #smote = SMOTE(ratio=1)
    #X_train_res, y_train_res = smote.fit_sample(X_train, y_train)
    print('Start Search')
    svm= SVC(class_weight='balanced', probability=True)
    pipe = Pipeline([('svm', svm)])
    param_grid = {
        'svm__C': [0.001, 0.01, 0.1, 1, 10, 100]}
    grid_search = GridSearchCV(estimator=pipe, param_grid=param_grid, n_jobs=6, cv=3, scoring='neg_log_loss', verbose=5)
    grid_search.fit(X_train, y_train)
    clf = grid_search.best_estimator_
    print('clf', clf)
    print('best_score', grid_search.best_score_)
    y_pred = clf.predict(X_test)
    check_pred = clf.predict(X_train)
    target_names = ['Not delinq', 'Delinq']
    print(classification_report(y_test, y_pred, target_names=target_names))
    conf_mat = confusion_matrix(y_test, y_pred)
    plt.figure()
    plot_confusion_matrix(conf_mat, classes=target_names,
                      title='Confusion matrix, without normalization')
    plt.show()
    return clf, clf.predict(X_train_res), y_pred

In [ ]:
clf, score, mat = complexity_svc(train, target)

In [ ]:
def train_size_svc(train=None, target=None, size=0):
    d = {'train': [], 'cv set': [], 'test': []}
    training_features, test_features, \
    training_target, test_target, = train_test_split(train, target, test_size=0.33, random_state=778)
    for size in np.arange(0.1, 1, 0.1):
        X_train, X_val, y_train, y_val = train_test_split(training_features, training_target, train_size=size)
        print('size', size)
        print('start')
        clf = SVC(verbose=True)
        clf.fit(X_train, y_train)
        d['train'].append(f1_score(y_train, clf.predict(X_train), average='weighted'))
        d['cv set'].append(f1_score(y_val, clf.predict(X_val), average='weighted'))
        d['test'].append(f1_score(test_target, clf.predict(test_features), average='weighted'))
        print('end')

    return d

In [ ]:
all_data = get_all_data()
train, target = process_data(all_data)
df = df = pd.DataFrame.from_dict(train_size_svc(train=train, target=target))
df.plot()
plt.show()

## KNN

In [6]:
def train_size_knn(train=None, target=None, size=0):
    d = {'train': [], 'cv set': [], 'test': []}
    training_features, test_features, \
    training_target, test_target, = train_test_split(train, target, test_size=0.33, random_state=778)
    for size in np.arange(0.1, 1, 0.1):
        print('size', size)
        X_train, X_val, y_train, y_val = train_test_split(training_features, training_target, train_size=size)
        print('start')
        start_time = time.time()
        clf = KNeighborsClassifier(n_neighbors=5)
        clf.fit(X_train, y_train)
        print('KNN took', time.time() - start_time, 'to run')
        d['train'].append(f1_score(y_train, clf.predict(X_train), average='weighted'))
        d['cv set'].append(f1_score(y_val, clf.predict(X_val), average='weighted'))
        d['test'].append(f1_score(test_target, clf.predict(test_features), average='weighted'))
        print('end')

    return d

In [9]:
all_data = get_all_data()
train, target = process_data(all_data)
df = pd.DataFrame.from_dict(train_size_knn(train=train, target=target))
merge_dict(df)
print(df)

sample_orig_2016.txt
(203642,)
size 0.1
start


C:\Users\jehun\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


KNN took 0.3795289993286133 to run


C:\Users\jehun\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


end
size 0.2
start
KNN took 1.6685278415679932 to run
end
size 0.3
start
KNN took 6.546721458435059 to run
end
size 0.4
start
KNN took 12.989205360412598 to run
end
size 0.5
start
KNN took 23.61362075805664 to run
end
size 0.6
start
KNN took 31.310216426849365 to run
end
size 0.7
start
KNN took 49.789954662323 to run
end
size 0.8
start
KNN took 69.1616063117981 to run
end
size 0.9
start
KNN took 89.00938153266907 to run
end


AttributeError: 'str' object has no attribute 'items'

## Notes 

1. We had 519 variables and the authors had 294. But to be sure, we don't have a greater number of features compared to the authors. I think this is just an artifact of our different implementations as the authors do have more data than us.

2. ROC curves are typically used in binary classification to study the output of a classifier. In order to extend ROC curve and ROC area to multi-class or multi-label classification, it is necessary to binarize the output as we had done. 